<a href="https://colab.research.google.com/github/Sergio-Per/MY-GUI-experiments/blob/main/MY_BOOTCAMP_FINAL_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

# **Time-Series Analysis with Python - Statistics, Predictions and GUIs**

#### by Sérgio Pereira, April 2022
---


A time-series is used in this project to code in python and explore a comprehensive set of statistical analysis, modeling, and prediction tools. These, combined with Interactive GUIs, allow the user to perform different actions on the data. For example, the user can select time periods or specific days and statistical parameters are computed and visualized; in the predictive models changes in hyperparameters are shown in the prediction curve plotted.

This notebook is expected to have some regular development at least for two reasons: (i) because the topic of time series is interesting and important in so many fields; (ii) this kind of notebook, with interactive widgets, convinced me on the interest of these kind of GUIs for educational purposes in mathematics, statistics, programming, or science in general.
A time-series is a sequence of measurements from a system that varies in time. Usually, these information points are collected at adjacent time-spaces, thus there is a relation between observations, proportional or not. This is a characteristic of time-series that differentiates them from other data. For coding purposes, a time-series is a set of data indexed by time.
Use case: Hourly data of Black Carbon (BC) mass concentration measurements from a small city (60000 in) are used throughout this notebook. Thus, this is a univariate time series, a sequence of measurements of the same variable. BC aerosols emerge in the atmosphere mainly produced by anthropogenic activities, with vehicles, industry, and biomass burning being the most important sources.
Section 1 deals essentially with descriptive statistics and main features of the data; in section 2 some further analysis is made, including the time-series autocorrelation and stationarity. Section 3 introduces the frequency domain with spectral analysis and shows some features of Fourier analysis. Section 4 brings machine learning techniques, and different predictive models are implemented and applied in the time-series.



<div class="alert alert-block alert-success">

- **Importing python libraries**



</div>

In [2]:
#@title
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy as sp
from scipy import stats

from matplotlib.gridspec import GridSpec
import matplotlib.patches as patches
%matplotlib inline

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from pandas.plotting import autocorrelation_plot,lag_plot

from statsmodels.tsa.stattools import adfuller,kpss#,range_unit_root_test,zivot_andrews

from scipy.fftpack import fft,fftfreq,ifft

import datetime

#from google.colab import files
import io

from __future__ import print_function




import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import ipywidgets as widgets
from ipywidgets import Layout, GridspecLayout, Box, VBox, HBox, Button, HTML
from ipywidgets import Image, TwoByTwoLayout, Label, Play
from ipywidgets import interact, interactive, fixed, interact_manual

- **Importing the file with data**
- **Importing the file with the data and some data processing**

In [5]:
#@title

url = 'https://raw.githubusercontent.com/Sergio-Per/MY-GUI-experiments/main/black_carbon.csv'
bc = pd.read_csv(url, na_values=('--'), parse_dates=["Datetime"])

bc= bc.set_index('Datetime')


bc['Year'] = pd.DatetimeIndex(bc.index).year
bc['Month'] = pd.DatetimeIndex(bc.index).month
bc['Day'] = pd.DatetimeIndex(bc.index).day
bc['Hour'] = pd.DatetimeIndex(bc.index).hour
bc['Weekday'] = pd.DatetimeIndex(bc.index).weekday

bc["BC"].fillna(bc["BC"].mean(), inplace=True)

bc.BC[bc.BC < 0.01] = 0.01

hourly = bc[['BC','Year', 'Month', 'Day', 'Hour', 'Weekday']].copy()




daily = hourly.resample("D").mean()
daily_sd = hourly.resample("D").std()


weekly = hourly.resample("W").mean()
weekly_sd = hourly.resample("W").std()

monthly = hourly.resample("M").mean()
monthly_sd = hourly.resample("M").std()

quarterly = hourly.resample("Q").mean()
quarterly_sd = hourly.resample("Q").std()


yearly =hourly.resample("Y").mean()
yearly_sd = hourly.resample("Y").std()


samples_list = [hourly, daily, weekly, monthly, quarterly, yearly]

# **1. Descriptive Statistics**

*   Year, month and day of observations can be selected, and the statistics and plots are updated accordingly.
*   The dataset is displayed for the time selected.
*   A table with statistical parameters (e.g., mean, quantiles, skewness) including a normality test.
*   Temporal evolution of BC and its daily cycle are displayed.   
*   Plots of the distribution of values (histogram, kernel density estimate and boxenplot).
*   Normality test is included.  


   

In [6]:
#@title

import warnings
warnings.filterwarnings('ignore')


#-------------------------------------------------------------------------------
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):   # returns list of unique values, sorted (year, month, day, etc)  
  unique=array.unique().tolist()
  unique.sort()
  unique.insert(0, ALL)
  return unique
#-------------------------------------------------------------------------------


output = widgets.Output()

plot_output =widgets.Output()

dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_ALL(hourly.Year), description='YEAR')
dropdown_month = widgets.Dropdown(options = unique_sorted_values_plus_ALL(hourly.Month), description='MONTH')
dropdown_day = widgets.Dropdown(options = unique_sorted_values_plus_ALL(hourly.Day), description='DAY')



def common_filtering(Year, Month, Day):

   
  output.clear_output()

  plot_output.clear_output()

  
  
  if (Year== ALL) & (Month==ALL) & (Day==ALL):
    common_filter = hourly
  
  elif (Year== ALL) & (Month==ALL):
    common_filter = hourly[hourly.Day == Day]

  elif (Year== ALL) & (Day==ALL):
    common_filter = hourly[hourly.Month == Month]

  elif (Month== ALL) & (Day==ALL):
    common_filter = hourly[hourly.Year == Year]       

  elif (Year== ALL):
    common_filter = hourly[(hourly.Month == Month) & (hourly.Day == Day)]

  elif (Month== ALL):
    common_filter = hourly[(hourly.Year == Year) & (hourly.Day == Day)]

  elif (Day==ALL):
    common_filter = hourly[(hourly.Year == Year) & (hourly.Month == Month)]
    
  else:
    common_filter = hourly[(hourly.Year == Year) & (hourly.Month == Month) & (hourly.Day == Day)]


  with output:
    display(common_filter)
  
#-------------------------------------------------------------------------------------------

  with plot_output:

    fig = plt.figure(figsize=(16,10),facecolor='gainsboro')
    fig.tight_layout()
    ax1 = plt.subplot2grid((3, 3), (0, 0), colspan=3)
  #  ax6 = plt.subplot2grid((3, 3), (0, 2))
    ax2 = plt.subplot2grid((3, 3), (1, 0), colspan=2)
    ax3 = plt.subplot2grid((3, 3), (1, 2), rowspan=2)
    ax4 = plt.subplot2grid((3, 3), (2, 0))
    ax5 = plt.subplot2grid((3, 3), (2, 1))


#-------------------------------------------------------------------------------------------

    ax1.set_title('Time Series   Day/Month/Year : '+ str(Day) + '/' + str(Month) + '/' + str(Year), y=0.8)

    ax1.plot(common_filter['BC'], c='black')
    ax1.fill_between(common_filter.index,
                     common_filter['BC'].mean()-common_filter['BC'].quantile(0.5), 
                     common_filter['BC'].mean()+common_filter['BC'].quantile(0.5),alpha=0.3)    
    ax1.axhline(common_filter['BC'].median(), alpha=0.7) 

    ax1.set_ylabel('BC (ugm-3)')
#-------------------------------------------------------------------------------------------

    ax2.set_title('Daily Cycle   Day/Month/Year : '+ str(Day) + '/' + str(Month) + '/' + str(Year),y=0.8)

    ax2.plot(common_filter.groupby('Hour')['BC'].mean(), 'o-', c='black', )
    ax2.fill_between(common_filter.groupby('Hour')['BC'].mean().index,
                     common_filter.groupby('Hour')['BC'].mean()-common_filter.groupby('Hour')['BC'].quantile(0.5), 
                     common_filter.groupby('Hour')['BC'].mean()+common_filter.groupby('Hour')['BC'].quantile(0.5),alpha=0.2)
    
    ax2.set_xlim(0,24)
    ax2.set_ylim(-0.2,6)
    ax2.set_ylabel('BC (ugm-3)')
#-------------------------------------------------------------------------------------------

    with sns.axes_style("whitegrid", {"axes.facecolor": "1"}):

      norm_stat, norm_p = stats.normaltest(common_filter['BC'])
      if norm_p < 0.05:  # null hypothesis: x comes from a normal distribution
        ax5.set_title('Normal Test: Failed', c='red', y=0.8)
      else:
        ax5.set_title('The distribution is Gaussian', y=0.8)
      #ax5.set_title('Normal Test '+ str())


      sns.kdeplot(common_filter['BC'], color='c',shade=False, ax=ax5)
      sns.distplot(common_filter['BC'], ax=ax5)
      y = stats.lognorm .pdf(np.linspace(0.1,10,100), 1.0462153447717188, 0.7452804722939874)
      ax5.axvline(common_filter['BC'].mean(),linestyle='-.', linewidth = 1, alpha = 1, c='black')
      ax5.axvline(common_filter['BC'].median(),linestyle='-.', linewidth = 1, alpha = 1, c='black')

      ax5.set_xlim(-1,common_filter['BC'].mean()+5)
      ax5.set_ylabel('BC (ugm-3)')

#-------------------------------------------------------------------------------------------
    
    column_labels=["Column 1"]
    df=pd.DataFrame([round(common_filter['BC'].count(),2),
                     round(common_filter['BC'].mean(),2),
                     round(common_filter['BC'].std(),2),
                     round(common_filter['BC'].min(),2),
                     round(common_filter['BC'].quantile(0.01),2),
                     round(common_filter['BC'].quantile(0.05),2),
                     round(common_filter['BC'].quantile(0.25),2),
                     round(common_filter['BC'].median(),2),
                     round(common_filter['BC'].quantile(0.75),2),
                     round(common_filter['BC'].quantile(0.95),2),
                     round(common_filter['BC'].quantile(0.99),2),
                     round(common_filter['BC'].max(),2),
                     round(common_filter['BC'].quantile(0.75)-common_filter['BC'].quantile(0.25),2),
                     round(common_filter['BC'].skew(),2), round(common_filter['BC'].kurtosis(),2),
                     round(norm_stat,2),norm_p],
                    columns=column_labels)
  
  
    ax3.set_title('Summary Statistics', fontsize=16, y = 0.95)
    
    ax3.axis('off')
    ax3.table(cellText=df.values, colLabels=['BC'],  
              rowLabels=['N Data',"Mean","Std",'Min','P1','P5' ,'P25','Median',
                         'P75','P95', 'P99','Max', 'IQR', 'Skew', 'Kurt', 
                         'Norm Stat','Norm p' ], 
              rowColours =["lightgrey"]*len(df) , colColours =["lightgrey"],
              loc='center',bbox = (0.2, 0, .6, .9))
    
    
#-------------------------------------------------------------------------------------------
    ax4.grid(True, linestyle='-.', linewidth = 0.5, alpha = 1, color='grey')
    sns.boxenplot(y=common_filter['BC'], color='royalblue', ax=ax4)
    ax4.set_ylabel('BC (ugm-3)')
#-------------------------------------------------------------------------------------------




#-------------------------------------------------------------------------------------------


#-------------------------------------------------------------------------------------------
    
    plt.show()


def dropdown_year_eventhandler(change):
  common_filtering(change.new, dropdown_month.value,dropdown_day.value)

def dropdown_month_eventhandler(change):
  common_filtering(dropdown_year.value, change.new,dropdown_day.value)  

def dropdown_day_eventhandler(change):
  common_filtering(dropdown_year.value, dropdown_month.value, change.new)

dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_month.observe(dropdown_month_eventhandler, names='value')
dropdown_day.observe(dropdown_day_eventhandler, names='value')



input_widgets = widgets.HBox([dropdown_year, dropdown_month, dropdown_day])


tab = widgets.Tab([output, plot_output])
tab.set_title(0, 'DATASET')
tab.set_title(1, 'STATS')
tab.set_title(2, 'TIMESERIES')


dashboard = widgets.VBox([input_widgets, tab])
display(dashboard)

# **2. Autocorrelation, Stationarity, Resampling**

The temporal structure adds an order to the observations in a time series. This means that important assumptions might need to be assessed prior to further analysis. Stationarity is a good example. Many time series techniques (notably statistical modeling methods) are based on the expectation of the time series being stationary; the summary statistics of observations have to be consistent. In time series jargon this assumption can be easily transgressed by the addition of a trend, of seasonality, or other time-dependent structures.

The autocorrelation function (ACF) for a series gives correlations between the series  and lagged values of the series for lags of 1 and n>1. The ACF can be used to identify the possible structure of time series data.
To compute serial correlation, we can shift the time series by an interval called a lag, and then compute the correlation of the shifted series with the original.

- The time resolution can be selected as well as the number of lags. The range of years under analysis can also be selected.

- The time series (based on daily values) is displayed. The shaded area includes $\pm$ 1 standard deviation of the mean (median is included in the plot. 

- The differenced time series, shifted by a number of n lags .

- the autocorrelation functions for the time series and the time series shifted by a lag of n days. The value for the lag can be selected. 

- The distribution of the values of the lag(n) series

- Results for the Dickey-Fuller unit root test for stationarity


In [7]:
#@title
from sklearn import linear_model

def my_figure1(n_days=14, yi=2007, yf=2012, dt_diff=1):


    ts1 = daily.BC[str(yi):str(yf)]
    ts2 = daily.BC[str(yi):str(yf)].resample(str(n_days)+'D').mean()
    ts3 = daily.BC[str(yi):str(yf)].resample(str(n_days)+'D').std()
    ts4 = daily.BC[str(yi):str(yf)].resample(str(n_days)+'D').median()
    
    ts_diff = ts2 - ts2.shift(dt_diff)
    ts_diff.dropna(inplace = True)
    
#------------------------------------------------------------------------------

    # X = ts2
    # y = ts2.shift(dt_diff).fillna(ts2.mean()) 

    # model = LinearRegression(fit_intercept=True)

    # model.fit(X[:, np.newaxis], y) 
    # Xfit = ts2 
    # yfit = model.predict(Xfit[:, np.newaxis])

    # model = LinearRegression(fit_intercept=True)

    # model.fit(X, y) 

    # yfit = model.predict(X)




#------------ adfuller test ----------------------------  


    af = adfuller(ts_diff, regression = 'ctt', autolag = 'AIC')

    # print('ADF Statistic: %f' % af[0])
    # print('p-value: %f' % af[1])
    # print('Critical Values:')
    # for key, value in af[4].items():
	  #   print('\t%s: %.3f' % (key, value))
    
    dfoutput = pd.Series(af[0:4], index = ['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
   
    for key,value in af[4].items():
        dfoutput['Critical Value (%s)'%key] = value
        
    # print('-------------------'*3)    
    # print('Results of Dickey-Fuller Test:')
    # print(dfoutput)
    # print('-------------------'*3)

#------------ Plotting time series----------------------------


    # fig = plt.figure(figsize=(18,12),facecolor='gainsboro')
    
    # ax1 = plt.subplot2grid((3, 3), (0, 0), colspan=2)
    # ax6 = plt.subplot2grid((3, 3), (0, 2))
    # ax2 = plt.subplot2grid((3, 3), (1, 0), colspan=2)
    # ax3 = plt.subplot2grid((3, 3), (1, 2), rowspan=2)
    # ax4 = plt.subplot2grid((3, 3), (2, 0))
    # ax5 = plt.subplot2grid((3, 3), (2, 1))

    fig = plt.figure(figsize=(18,12),facecolor='gainsboro')
    fig.tight_layout()
    ax1 = plt.subplot2grid((3, 3), (0, 0), colspan=2)
    ax2 = plt.subplot2grid((3, 3), (0, 2))
    ax3 = plt.subplot2grid((3, 3), (1, 0), colspan=2)
    ax4 = plt.subplot2grid((3, 3), (1, 2))
    ax5 = plt.subplot2grid((3, 3), (2, 0))
    ax6 = plt.subplot2grid((3, 3), (2, 1))
    ax7 = plt.subplot2grid((3, 3), (2, 2))

#-------------------------------------------------------------------------------------------------------
    ax1.set_title('Time Series', size=14, y=0.8)

    ax1.plot(ts1,c='black',alpha=0.22, label = 'hourly values, initial data')
    ax1.plot(ts2, c='darkblue',linewidth = 2, label = 'resampled values, mean') 
    ax1.plot(ts4, c='blue',linewidth = 2, alpha =1, label = 'resampled values, median')
    ax1.fill_between(ts2.index,ts2-ts3, ts2+ts3, alpha = 0.4,  label = 'std')
    ax1.plot(ts2+ts3, c = 'darkblue', linewidth = .5, alpha=1)
    ax1.plot(ts2-ts3, c = 'darkblue', linewidth = .5, alpha=1)
    
    ax1.set_ylim(0,(ts2+ts3).max()+1)    
    #ax1.legend( loc='upper left', shadow=True, fontsize='medium')
    
    ax1.set_ylabel('BC (ugm-3)')


#-------------------------------------------------------------------------------------------------------
    ax2.set_title('Autocorrelation', size=12, loc='left')
    ax2.set_title('BC(t)', size=12, loc='right')
    #ax2.set_facecolor('whitesmoke')
    autocorrelation_plot(ts2,ax=ax2)
#     plot_acf(ts2, lags = lags ,label = 'acf',ax = ax2)
#     plot_pacf(ts2, lags = lags ,method='ywm',label = 'pacf',ax = ax2)
#    ax2.legend( loc='best', shadow=True, fontsize='medium')
    ax2.set_xlim(0,50) 
    ax2.set_ylim(-1.5,1.5)
    ax2.set_xlabel('')

#-------------------------------------------------------------------------------------------------------

    #ax2.set_title('Time series \n Hourly BC measurements', size=16)
    
    ax3.plot(ts_diff, c='darkblue',linewidth = 2, label = 'diff')
    ax3.set_title('Time Series  BC(t)-BC(t-lag)', size=12)
    ax3.set_ylim(-1.5,1.5)
    ax3.set_ylabel('Diff (ugm-3)')
    #ax1.set_facecolor('whitesmoke')
    #ax1.legend( loc='best', shadow=True, fontsize='medium')

#-------------------------------------------------------------------------------------------------------

    ax4.set_title('Autocorrelation  ', loc='left',size=12)
    ax4.set_title('BC(t)-BC(t-lag)', loc='right',size=12)
    #ax3.set_facecolor('whitesmoke')
    autocorrelation_plot(ts_diff, c= 'darkblue', linewidth = 2, ax=ax4)
#     plot_acf(ts_diff, lags = lags ,label = 'acf', c = 'green',ax = ax4)
#     plot_pacf(ts_diff, lags = lags ,method='ywm',label = 'pacf',ax = ax4)
#    ax4.legend( loc='best', shadow=True, fontsize='medium')
    ax4.set_xlim(0,50)
    ax4.set_ylim(-1,1)
    ax4.set_xlabel('')
#-------------------------------------------------------------------------------------------------------
    X = ts2
    y = ts2.shift(dt_diff).fillna(ts2.mean()) 

    model = LinearRegression(fit_intercept=True)

    model.fit(X[:, np.newaxis], y) 
    Xfit = ts2 
    ypred = model.predict(Xfit[:, np.newaxis])
    ax5.set_title(' BC(t) vs BC(t-lag): ', size=12)

    ax5.scatter(X,y)
    ax5.plot(Xfit[:, np.newaxis],ypred, linewidth = 2.5,c='red')



#-------------------------------------------------------------------------------------------------------

    # for key, value in af[4].items():
	  #   print('\t%s: %.3f' % (key, value))

    column_labels=["Column 1"]
    df=pd.DataFrame([af[0], af[1]],columns=column_labels)
    df2 = pd.DataFrame(af[4].values(),columns=column_labels)
    result = pd.concat([df,df2])
    
    # for key,value in af[4].items():
    #     dfoutput['Critical Value (%s)'%key] = value
    # ax2.axis('tight')
    # ax1.set_title('Year/Month/Day :  '+ str(Year) + '/' + str(Month) + '/' + str(Day))
    ax6.set_title('Stationarity test', size=14, y=0.90)
    
    ax6.axis('off')
    ax6.table(cellText=result.values, colLabels=['Dickey-Fuller Test'],
              rowLabels=['ADF Statistic:','p-value:', '1%','5%','10%' ], 
              rowColours =["lightgrey"]*len(result) , colColours =["lightgrey"], loc='center',bbox = (0.2,0,0.6,.8))
#-------------------------------------------------------------------------------------------------------
    ax7.set_title('BC(t)-BC(t-lag)', loc='right',size=12)
    ax7.set_title('Distribution', loc='left',size=12)  
    sns.kdeplot(ts_diff, color='c',shade=False, ax=ax7)
    sns.distplot(ts_diff, ax=ax7)
    ax7.set_ylabel('Diff (ugm-3)')
    ax7.set_xlim(-2,2)
#-------------------------------------------------------------------------------------------------------



yi = widgets.IntSlider(value=2007,min=2007,max=2012,step=1,description='Yi : ',disabled=False,continuous_update=False,
                           orientation='horizontal', readout=True, readout_format='d',
                           style=dict(handle_color = 'lightblue'))
#yi.style.handle_color = 'lightblue'


yf = widgets.IntSlider(value=2012,min=2007,max=2012,step=1,description='Yf : ',disabled=False,continuous_update=False,
                           orientation='horizontal', readout=True, readout_format='d')


n_days = widgets.IntSlider(value=14,min=2,max=180,step=2,description='WINDOW : ',disabled=False,continuous_update=False,
                           orientation='horizontal', readout=True, readout_format='d')

#lags = widgets.IntSlider(value=50,min=2,max=100,step=1,description='Lag : ',disabled=False,continuous_update=False,
#                           orientation='horizontal', readout=True, readout_format='d')

dt_diff = widgets.IntSlider(value=1,min=0,max=60,step=1,description='LAG : ',disabled=False,continuous_update=False,
                           orientation='horizontal', readout=True, readout_format='d')

#from ipywidgets import TwoByTwoLayout


#interact(my_figure1 , n_days=n_days, yi=(2007,2012), yf=(2007,2012),  dt_diff = dt_diff);


# ui = widgets.HBox([p, d ,q,P,D,Q,s])

# out = widgets.interactive_output(my_figure_sarimax, {'p': p, 'd': d,'q': q,'P': P, 'D': D,'Q': Q,'s': s })

# display(out, ui)
out = widgets.interactive_output(my_figure1, {'yi': yi, 'yf': yf, 'n_days': n_days, 'dt_diff': dt_diff})

#widgets.VBox([widgets.VBox([yi, yf, n_days,dt_diff]), out])

box1 = TwoByTwoLayout(top_left=yi,
               top_right=n_days,
               bottom_left=yf,
               bottom_right=dt_diff)

#box1 = Layout()
box1.width = '700px'
box1.justify = 'end'

box2 = VBox([box1, out])

display(box2)

# **3. Frequency Domain, Spectral Analysis, FFT, High-Pass/Low-Pass filters**

The fact that any time series can be written as a sum of sine and cosine waves is used to examine its periodic (cyclical) behavior. 

The Periodogram is used to identify the dominant periods (or frequencies) of the data (the minimum time resolution here is 1 hour). The peaks can be identified and related to the dominant periods. Notice the difference in the periodogram when the time resolution in the time series is changed.

The effect of high-pass (HP) and low-pass (LP) filters applied to the time series. The cut off values for the filters can be made to vary for better observing the effect. 


  



In [8]:
#@title
hourly = bc[(bc.index < '2011-04-24 12:00:00')].copy()
import warnings
warnings.filterwarnings('ignore')

def my_figure_FFT( n_hours = 1,play=1):

  t = np.arange(len(np.array(hourly.BC[:'2011'])))
  ts_resampled = hourly.BC[:'2011'].rolling(n_hours, min_periods=1).mean()

  #ts_resampled = hourly.BC[:'2011'].resample(str(n_hours)+'H').mean()



  sig_fft_original = fft(np.array(hourly.BC[:'2011']))
  sig_fft = fft(np.array(ts_resampled))
  sig_fft_filters = fft(np.array(hourly.BC[:'2011']))
    # copy the FFT results
  
    # obtain the frequencies using scipy function
  freq_original = fftfreq(len(hourly.BC[:'2011']), d=1)
  freq = fftfreq(len(np.array(ts_resampled)), d=1)

    # copy the FFT results
  sig_fft_filtered_low = sig_fft_filters.copy()
  sig_fft_filtered_high = sig_fft_filters.copy()
    # obtain the frequencies using scipy function
  freq_low = fftfreq(len(np.array(hourly.BC[:'2011'])), d=1)
  freq_high = fftfreq(len(np.array(hourly.BC[:'2011'])), d=1)
    # define the cut-off frequency
    #cut_off = 0.0006

    # low-pass filter 

  # sig_fft_filtered_low[(np.abs(freq_low)) > (cut_off_l)] = 0
    
  # sig_fft_filtered_high[(np.abs(freq_high)) < (cut_off_h)] = 0

    # get the filtered signal in time domain
  filtered_low = ifft(sig_fft_filtered_low)
  filtered_high = ifft(sig_fft_filtered_high)


  
  fig = plt.figure(figsize=(20,4),facecolor='gainsboro')
  fig.suptitle('FFT and Frequency content')
    
  ax1 = plt.subplot2grid((1, 4), (0, 0), colspan=2)
  ax2 = plt.subplot2grid((1, 4), (0, 2),colspan=2)

  #fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4),gridspec_kw={'width_ratios': [3, 1.5]})
  fig.set_facecolor('#E0E0E0')
  ax1.set_title('Time series with Resolution (hours) : ' + str(n_hours))
  ax1.plot(t,hourly.BC[:'2011'], c='black', alpha = 0.2)
  ax1.plot(t, ts_resampled, c='steelblue', alpha = 1)
  ax1.set_ylabel('BC(ugm-3)');
   
  ax2.plot(1/freq_original, np.abs(sig_fft_original),c='black')
  ax2.plot(1/freq, np.abs(sig_fft))
  ax2.axvline(1, 0, 0.15, linewidth = 3 , c='black')
  ax2.axvline(12, 0, 0.15, linewidth = 3 , c='black')
  ax2.axvline(24, 0, 0.15, linewidth = 3 , c='black')
  ax2.axvline(8760, 0, 0.15, linewidth = 3 , c='black')
  ax2.set_title('Periodogram')
  ax2.set_xticks([12, 24, 48, 72, 84, 168])
  #ax2.set_xlim(0, 200)
  #ax2.set_ylim(10, 10000)
  ax2.set_xlabel('Hour')
  ax2.set_ylabel('FFT Amplitude');
  ax2.set_yscale("log")
  ax2.set_xscale("log");
  ax2.set_ylim(1e-1,2e4)
  plt.show()





n_hours = widgets.IntSlider(value=1,min=1,max=10000,step=1,description='Window : ',disabled=False,continuous_update=False,
                            orientation='horizontal', readout=True, readout_format='d')


play = widgets.Play(
    value=1,
    min=1,
    max=100,
    step=1,
    interval=200,
    description="Press play",
    disabled=False)



slider = n_hours
widgets.jslink((play, 'value'), (slider, 'value'))
# box=widgets.HBox([play])
# display(box)
interact(my_figure_FFT , n_hours=n_hours,play=play )




#################################################################################
####################################################################################


def my_figure_Filters(cut_off_l = 1.7,cut_off_h = 0.001):



#plot the filtered signal


  t = np.arange(len(np.array(hourly.BC[:'2011'])))

  sig_fft_original = fft(np.array(hourly.BC[:'2011']))
  
  sig_fft_filters = fft(np.array(hourly.BC[:'2011']))
    # copy the FFT results
  
    # obtain the frequencies using scipy function
  freq_original = fftfreq(len(hourly.BC[:'2011']), d=1)
  

    # copy the FFT results
  sig_fft_filtered_low = sig_fft_filters.copy()
  sig_fft_filtered_high = sig_fft_filters.copy()
    # obtain the frequencies using scipy function
  freq_low = fftfreq(len(np.array(hourly.BC[:'2011'])), d=1)
  freq_high = fftfreq(len(np.array(hourly.BC[:'2011'])), d=1)
    # define the cut-off frequency
    #cut_off = 0.0006

    # low-pass filter 

  sig_fft_filtered_low[(np.abs(freq_low)) > (cut_off_l)] = 0
    
  sig_fft_filtered_high[(np.abs(freq_high)) < (cut_off_h)] = 0

    # get the filtered signal in time domain
  filtered_low = ifft(sig_fft_filtered_low)
  filtered_high = ifft(sig_fft_filtered_high)

  fig = plt.figure(figsize=(20,4),facecolor='gainsboro')
  fig.suptitle('High/Low Pass Filters')
    
  ax_low = plt.subplot2grid((1, 4), (0, 0), colspan=2)
  ax_high = plt.subplot2grid((1, 4), (0, 2),colspan=2)

  fig.set_facecolor('#E0E0E0') 
  ax_low.set_title('Low Pass Filter on the signal, cutoff = ' + str(round(365*cut_off_l,6))+ '(day)')
  ax_low.plot(t,hourly.BC[:'2011'], c='black', alpha = 0.1)
  ax_low.plot(t, filtered_low, c = 'steelblue', linewidth = 1.5)
  ax_low.set_xlim(-500,35000)
  ax_low.set_ylim(0, 16)
    
  ax_low.set_xlabel('Time (h) ')
  ax_low.set_ylabel('Amplitude')

  ax_high.set_title('High Pass Filter on the signal, cutoff = ' + str(round(365*cut_off_h,6))+ '(day)')
  ax_high.plot(t,hourly.BC[:'2011'], c='black', alpha = 0.1)
  ax_high.plot(t, filtered_high, c = 'steelblue',linewidth = 1.5)
  ax_high.set_xlim(-500,35000)
  ax_high.set_ylim(0, 16)
  # ax_high.set_yscale("log")  
  ax_high.set_xlabel('Time (h) ')
 # ax_high.set_ylabel('Amplitude')




cut_off_l = widgets.FloatLogSlider(value=1.7,base=10,min=-5, max=1, step=0.001, description='LP cut', 
                                   orientation='horizontal', readout_format='.5f')

cut_off_h = widgets.FloatLogSlider(value=0.001,base=10,min=-6, max=-0.30, step=0.01, description='HP cut', 
                                   orientation='horizontal', readout_format='.5f')

# interact(my_figure_Filters ,cut_off_l=cut_off_l,cut_off_h=cut_off_h)


out = widgets.interactive_output(my_figure_Filters, {'cut_off_l': cut_off_l, 'cut_off_h': cut_off_h})

box = widgets.VBox([widgets.VBox([cut_off_l, cut_off_h]), out])
display(box)


# box=widgets.HBox([play])
# display(box)



interactive(children=(IntSlider(value=1, continuous_update=False, description='Window : ', max=10000, min=1), …

# **4. Time series forecasting/Machine Learning**

The forecasting process consists of predicting future values of a time series, either by modeling the series based on its previously observed patterns (autoregressive) or by using other external variables. The former approach is to be adopted with the univariate BC measurements.

Three different models are briefly explored: ARIMA/SARIMAX as well as Random Forest and Lasso regressors were used to create forecasters, including grid search with backtesting (cross-validation method applied to previous periods) to identify the best combination of lags and hyperparameters for the models.
In the following subsections each model is trained, and the best combination of parameters is selected and presented. The predictions can be computed and visualized with these best parameters or other values can be selected with the predictions being updated accordingly.

Briefly, Lasso is a Linear Model trained with L1 regularization and it estimates sparse coefficients. The alpha parameter controls the degree of sparsity of the estimated coefficients. A random forest is a meta estimator that fits a number of classifying decision trees on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is controlled with the max_samples parameter if bootstrap=True (default), otherwise the whole dataset is used to build each tree. SARIMAX (Seasonal Auto-Regressive Integrated Moving Average with eXogenous factors) is an extension of the ARIMA class of models. ARIMA models compose 2 parts: the autoregressive term (AR) and the moving-average term (MA)


In [10]:
pip install skforecast


Defaulting to user installation because normal site-packages is not writeable
  Using cached skforecast-0.4.3-py2.py3-none-any.whl (87 kB)
Note: you may need to restart the kernel to use updated packages.


**Importing Libraries**

In [11]:
#@title
# Modeling and Forecasting
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster


from skforecast.model_selection_statsmodels import grid_search_sarimax

from joblib import dump, load
from sklearn.svm import SVR
# Warnings configuration
# ==============================================================================
import warnings
# warnings.filterwarnings('ignore')

## **4.1. Parameter grid search and prediction with Random Forest Regressor**








In [12]:
#@title
## Hyperparameter Grid search RandomForestRegressor
# ==============================================================================

from tqdm import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)


data = monthly.BC.reset_index(drop=True)
#data = hourly.BC.resample("M").median().reset_index(drop=True)
steps = 12

data_train = data[:-steps]
data_test  = data[-steps:]


forecaster = ForecasterAutoreg(regressor = RandomForestRegressor(random_state=123),lags = 6) # This value will be replaced in the grid search


# Lags used as predictors
lags_grid = [1,2,3,4,5,6,12]

# Regressor's hyperparameters
param_grid = {'n_estimators': [100, 500], 'max_depth': [1, 2, 3]}

results_grid_rf = grid_search_forecaster(
                        forecaster         = forecaster,
                        y                  = data_train,
                        param_grid         = param_grid,
                        lags_grid          = lags_grid,
                        steps              = steps,
                        refit              = True,
                        metric             = 'mean_squared_error',
                        initial_train_size = int(len(data_train)*0.5),
                        fixed_train_size   = False,
                        return_best        = True,
                        verbose            = False
               )



results_grid_rf[:5]

Number of models compared: 42
`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4 5 6] 
  Parameters: {'max_depth': 1, 'n_estimators': 500}
  Backtesting metric: 0.06867171063479327



,lags,params,metric,max_depth,n_estimators
31,"[1, 2, 3, 4, 5, 6]","{'max_depth': 1, 'n_estimators': 500}",0.068672,1,500
30,"[1, 2, 3, 4, 5, 6]","{'max_depth': 1, 'n_estimators': 100}",0.068912,1,100
34,"[1, 2, 3, 4, 5, 6]","{'max_depth': 3, 'n_estimators': 100}",0.072445,3,100
32,"[1, 2, 3, 4, 5, 6]","{'max_depth': 2, 'n_estimators': 100}",0.073689,2,100
37,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","{'max_depth': 1, 'n_estimators': 500}",0.077030,1,500


In [13]:
#@title

def my_figure_RandomForestRegressor(lags= 6, max_depth = 1,
                                    n_estimators = 500):


# Create and train forecaster with the best hyperparameters
# ==============================================================================
  regressor = RandomForestRegressor(max_depth=max_depth, n_estimators=n_estimators, 
                                    bootstrap =True  , random_state=123)
  forecaster = ForecasterAutoreg(regressor = regressor,lags = lags)

  forecaster.fit(y=data_train)

  predictions = forecaster.predict(steps=steps)
  #print(predictions.head())
  error_mse_rforest = mean_squared_error(y_true = data_test,y_pred = predictions)

 #  print(f"Test error (mse): {error_mse_rforest}")

  # Plot
  # ==============================================================================
  fig, ax = plt.subplots(figsize = (10,5))
  fig.set_facecolor('#E0E0E0')
  ax.set_title('Random Forest \n error (mse): '+str(error_mse_rforest), y=0.85)
  #ax.set_facecolor('whitesmoke')
  ax.plot(data_train[:], 'o-', c='black' )
  ax.plot(data_test, 'o-',c='blue')
  #ax.plot(df_valid,  'o-', c='black')
  ax.plot(predictions, linewidth=4, c='red')   #predictions SARIMAX
    #ax.plot(res.predict('2012-03-31', '2012-05-30'),c='black')            
  ax.set_ylim(0,5)
  ax.set_ylabel('BC (mgm-3)');
  
  # ax.fill_between(
  #   predictions.index,
  #   predictions['lower_bound'],
  #   predictions['upper_bound'],
  #   color = 'red',
  #   alpha = 0.2,
  #   label = 'prediction_interval')


lags = widgets.IntSlider(value=6,min=1,max=12,step=1,description='LAGS ',
                         disabled=False,continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
max_depth = widgets.IntSlider(value=1,min=1,max=7,step=1,description='MAX DEP ', disabled=False,continuous_update=False,
                         orientation='horizontal', readout=True, readout_format='d')
n_estimators = widgets.IntSlider(value=500,min=1,max=1000,step=10,description='N EST ',
                                 disabled=False,continuous_update=False, orientation='horizontal', readout=True, readout_format='d')

# ui = widgets.HBox([lags, max_depth,n_estimators])

# #out = widgets.interactive_output(my_figure_RandomForestRegressor, {'lags': lags, 'max_depth': max_depth,'n_estimators': n_estimators })

# display(out, ui)



out = widgets.interactive_output(my_figure_RandomForestRegressor, 
                                 {'lags': lags, 'max_depth': max_depth,'n_estimators': n_estimators })

box = widgets.VBox([widgets.VBox([lags, max_depth, n_estimators]), out])

display(box)



## **4.2. Parameter grid search and prediction with Lasso model**


In [ ]:
#@title
data = monthly.BC.reset_index(drop=True)

steps = 12

data_train = data[:-steps]
data_test  = data[-steps:]



# Hyperparameter Grid search
# ==============================================================================
forecaster = ForecasterAutoregMultiOutput(
                    regressor = make_pipeline(StandardScaler(), Lasso(fit_intercept=True, random_state=123)),
                    steps     = 12,
                    lags      = 6 # This value will be replaced in the grid search
             )

# To access parameters of a scikitlearn pipeline the pattern:
# <name of the step>__<name of the parameter>.
param_grid = {'lasso__alpha': np.logspace(-5, 3, 10)}
lags_grid = [1,2,3,4,5,6,7,8,9,10,11, 12]

results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = data_train,
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 12,
                        refit       = True,
                        metric      = 'mean_squared_error',
                        initial_train_size = int(len(data_train)*0.5),
                        fixed_train_size   = False,
                        return_best = True,
                        verbose     = False
                )

results_grid.head()

Number of models compared: 120
`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12] 
  Parameters: {'lasso__alpha': 0.004641588833612777}
  Backtesting metric: 0.051006010312766865



,lags,params,metric,lasso__alpha
113,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]",{'lasso__alpha': 0.004641588833612777},0.051006,0.004642
114,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]",{'lasso__alpha': 0.03593813663804626},0.059218,0.035938
104,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",{'lasso__alpha': 0.03593813663804626},0.067586,0.035938
103,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",{'lasso__alpha': 0.004641588833612777},0.069165,0.004642
102,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",{'lasso__alpha': 0.0005994842503189409},0.071499,0.000599


In [ ]:
#@title
def my_figure_Lasso(alpha = 0.004642):


# Create and train forecaster with the best hyperparameters
# ==============================================================================
  # regressor = Lasso(max_depth=max_depth, n_estimators=n_estimators, random_state=123)
  # forecaster = ForecasterAutoreg(regressor = regressor,lags = lags)
  forecaster = ForecasterAutoregMultiOutput(
                    regressor = make_pipeline(StandardScaler(), Lasso(alpha=alpha, random_state=123)),
                    steps     = 12,
                    lags      = 18 
             )
  forecaster.fit(y=data_train)

  predictions = forecaster.predict(steps=steps)
  #print(predictions.head())
  error_mse_lasso = mean_squared_error(data_test,predictions)


 #  print(f"Test error (mse): {error_mse_rforest}")

  # Plot
  # ==============================================================================

  

  fig, ax = plt.subplots(figsize = (10,5))
  fig.set_facecolor('#E0E0E0')

  ax.set_title('Lasso \n error (mse): '+str(error_mse_lasso), y=0.85)
  #ax.set_facecolor('whitesmoke')
  ax.plot(data_train, 'o-', c='black' )
  ax.plot(data_test, 'o-',c='black')

  ax.plot(predictions, linewidth=4, c='red')   
            
  ax.set_ylim(0,5)
  ax.set_ylabel('BC mgm-3');


alpha = widgets.FloatLogSlider(value=0.004642,base=10,min=-7, max=5, step=0.01, description='alpha', 
                                   orientation='horizontal', readout_format='.5f')
interact(my_figure_Lasso, alpha=alpha );




interactive(children=(FloatLogSlider(value=0.004642, description='alpha', max=5.0, min=-7.0, readout_format='.…


## **4.3. Parameter grid search and prediction with ARIMA/SARIMAX models**

**(p,d,q)** - order of the model for the number of AR parameters, differences, and MA parameters.   
**(P,D,Q,s)** order of the seasonal component of the model for the AR parameters, differences, MA parameters, and periodicity.  
These parameters can be changed and the prediction is updated in the plot.

In [ ]:
#@title
## Hyperparameter Grid search RandomForestRegressor
# ==============================================================================
#from skforecast.model_selection_statsmodels import grid_search_sarimax

data = monthly.BC#.reset_index(drop=True)

steps = 12


data_train = data[:'2010-12-31']
data_val = data.loc['2010-12-31' : '2011-05-31']
data_test  = data['2011-05-31':]



param_grid = {'order': [(12, 0, 0), (12, 1, 0), (12, 1, 1), (12, 0, 1),(12,1,0)],
             'seasonal_order': [(0,0,0,12),(0,1,0,12),(1,1,0,8)],
             'trend': [None, 'n', 'c']}

results_grid = grid_search_sarimax(y = data.loc[:'2011-05-31'],param_grid = param_grid,
                                   initial_train_size = len(data_train),
                                   fixed_train_size = False,
                                   steps = 6,
                                   metric = 'mean_absolute_error',
                                   refit = True,
                                   verbose = False, fit_kwargs = {'maxiter': 300, 'disp': 0})

results_grid.head()


root       INFO  Number of models compared: 45


,params,metric,order,seasonal_order,trend
19,"{'order': (12, 1, 1), 'seasonal_order': (0, 0,...",0.172935,"(12, 1, 1)","(0, 0, 0, 12)",n
18,"{'order': (12, 1, 1), 'seasonal_order': (0, 0,...",0.172935,"(12, 1, 1)","(0, 0, 0, 12)",None
36,"{'order': (12, 1, 0), 'seasonal_order': (0, 0,...",0.185072,"(12, 1, 0)","(0, 0, 0, 12)",None
37,"{'order': (12, 1, 0), 'seasonal_order': (0, 0,...",0.185072,"(12, 1, 0)","(0, 0, 0, 12)",n
9,"{'order': (12, 1, 0), 'seasonal_order': (0, 0,...",0.185072,"(12, 1, 0)","(0, 0, 0, 12)",None


In [ ]:
#@title
import warnings
warnings.filterwarnings('ignore')

monthly = bc.resample("M").mean()

df_train = monthly.BC[:-15]
df_test = monthly.BC[-15:-12]
df_valid = monthly.BC[-12:]


def my_figure_sarimax(p=12,d=1,q=0, P=0, D=1, Q = 0 , s=12):
    

# -------------SARIMAX------------------------------------------------------------------- 
#------------------------------------------------------------------------------------------

  mod = sm.tsa.statespace.SARIMAX(df_train, order=(p,d,q), seasonal_order=(P,D,Q,s));
    
  res_sarimax = mod.fit(disp=True)
    
  print(res_sarimax)
    
    #print(res.summary())
  error_mse_sarimax = mean_squared_error(y_true = df_valid,y_pred = res_sarimax.predict('2011-05-31', '2012-04-30'))


#------------------------------------------------------------------------------------------

  fig, ax = plt.subplots(figsize = (10,5))
  fig.set_facecolor('#E0E0E0')
  ax.set_title('SARIMAX \n error (mse): '+str(error_mse_sarimax), y=0.85) 
  #ax.set_facecolor('whitesmoke')
  ax.plot(df_train, 'o-', c='black' )
  ax.plot(df_test, 'o-',c='black')
  ax.plot(df_valid,  'o-', c='black')
  ax.plot(res_sarimax.predict('2011-05-31', '2012-04-30'), linewidth=3, c='red')   #predictions SARIMAX
    #ax.plot(res.predict('2012-03-31', '2012-05-30'),c='black')            
  ax.set_ylim(0,5)
  ax.set_ylabel('BC(mgm-3');
  
  plt.show()



p = widgets.IntSlider(value=12,min=0,max=30,step=1,description='p ',disabled=False,continuous_update=False,
                           orientation='horizontal', readout=True, readout_format='d')
    
d = widgets.IntSlider(value=1,min=0,max=30,step=1,description='d ',disabled=False,continuous_update=False,
                           orientation='horizontal', readout=True, readout_format='d') 

q = widgets.IntSlider(value=0,min=0,max=30,step=1,description='q ',disabled=False,continuous_update=False,
                           orientation='horizontal', readout=True, readout_format='d')

P = widgets.IntSlider(value=0,min=0,max=30,step=1,description='P : ',disabled=False,continuous_update=False,
                           orientation='horizontal', readout=True, readout_format='d')
    
D = widgets.IntSlider(value=1,min=0,max=2,step=1,description='D ',disabled=False,continuous_update=False,
                           orientation='horizontal', readout=True, readout_format='d') 

Q = widgets.IntSlider(value=0,min=0,max=30,step=1,description='Q ',disabled=False,continuous_update=False,
                           orientation='horizontal', readout=True, readout_format='d')

s = widgets.IntSlider(value=12,min=0,max=30,step=1,description='s ',disabled=False,continuous_update=False,
                           orientation='horizontal', readout=True, readout_format='d')
    


#interact(my_figure_sarimax, p=p,d=d,q=q, P=P, D=D, Q = Q , s=s );



out = widgets.interactive_output(my_figure_sarimax, {'p': p, 'd': d, 'q': q,'P': P, 'D': D, 'Q': Q, 's':s})

box = widgets.HBox([widgets.VBox([p, d, q, P,D,Q,s]), out])

display(box)



In this exercise the predictions obtained with SARIMAX had the lowest mean squared error, 0.08, while Lasso and Random Forest had mse of 0.9 and 0.1, respectively. However the methodology used has plenty of room for improvements. These include the number of parameters being optimized with grid search. Also other models are able to be explored. 

# **5. References**




Matplotlib Tutorials: https://matplotlib.org/stable/tutorials/index.html

Numpy User Guide: https://numpy.org/doc/stable/user/index.html

Pandas Documentation: https://pandas.pydata.org/

Scikit-learn User Guide: https://scikit-learn.org/stable/user_guide.html

Scipy Documentation: https://scipy.github.io/devdocs/index.html

Seaborn Tutorial: https://seaborn.pydata.org/tutorial.html

Skforecast User guides: https://joaquinamatrodrigo.github.io/skforecast/0.4.3/index.html


